In [1]:
import numpy as np
from datetime import datetime

# 수치미분 함수
def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = float(tmp_val) - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad


# sigmoid 함수
def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [2]:
# LogicGate Class

class LogicGate:
    
    def __init__(self, gate_name, xdata, tdata, lr, cnt):  # xdata, tdata => numpy.array(...)
        
        self.name = gate_name
        
        self.xdata = xdata.reshape(4, 2)
        self.tdata = tdata.reshape(4, 1)
        
        # 가중치 W, 바이어스 b 초기화 - 일반적인 방법으로 변경 필요
        self.W = np.random.rand(self.xdata.shape[1], 1)  # weight, 2 X 1 matrix
        self.b = np.random.rand(1)
                        
        # 학습률 learning rate 초기화
        self.learning_rate = lr
        
        # 반복횟수 초기화
        self.iteration_count = cnt
        
    # 손실함수
    def loss_func(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )      
    
    # 손실 값 계산
    def error_val(self):
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z = np.dot(self.xdata, self.W) + self.b
        y = sigmoid(z)
    
        # cross-entropy 
        return  -np.sum( self.tdata*np.log(y + delta) + (1-self.tdata)*np.log((1 - y)+delta ) )

    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        step_count = int(self.iteration_count * 0.1)
        
        f = lambda x : self.loss_func()
        
        print("Initial error value = ", self.error_val())
        
        start_time = datetime.now()
        
        for step in  range(self.iteration_count):
            
            self.W -= self.learning_rate * numerical_derivative(f, self.W)
    
            self.b -= self.learning_rate * numerical_derivative(f, self.b)
    
            if (step % step_count == 0):
                print("step = ", step, "error value = ", self.error_val())
                
        end_time = datetime.now()
        
        print('\nElapsed Time => ', end_time - start_time)
                
    # 미래 값 예측 함수
    def predict(self, input_data):
        
        z = np.dot(input_data, self.W) + self.b
        y = sigmoid(z)
    
        if y >= 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result
    
    # 정확도 예측 함수
    def accuracy(self, test_xdata, test_tdata):
        
        matched_list = []
        not_matched_list = []
        
        for index in range(len(xdata)):
            
            (real_val, logical_val) = self.predict(test_xdata[index])
            
            if logical_val == test_tdata[index]:
                matched_list.append(index)
            else:
                not_matched_list.append(index)
                
        accuracy_val = len(matched_list) / len(test_xdata)
        
        return accuracy_val

In [3]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1])

AND_obj = LogicGate("AND_GATE", xdata, tdata, 1e-2, 20001)

AND_obj.train()

Initial error value =  4.348396102098628
step =  0 error value =  4.293808879493962
step =  2000 error value =  0.6582455476568896
step =  4000 error value =  0.389551285961387
step =  6000 error value =  0.2745380585977655
step =  8000 error value =  0.21103332504033898
step =  10000 error value =  0.17097042539057633
step =  12000 error value =  0.14348177207941124
step =  14000 error value =  0.12349260548779639
step =  16000 error value =  0.10832374235811619
step =  18000 error value =  0.09643094205303257
step =  20000 error value =  0.0868631611814432

Elapsed Time =>  0:00:10.269092


In [4]:
# AND Gate prediction
print(AND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 0, 0, 1])

accuracy_ret = AND_obj.accuracy(test_xdata, test_tdata)

print("Accuracy => ", accuracy_ret)

AND_GATE 

Accuracy =>  1.0


In [5]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])

OR_obj = LogicGate("OR_GATE", xdata, tdata, 1e-2, 20001)

OR_obj.train() 

Initial error value =  1.7688412815377137
step =  0 error value =  1.764851140800651
step =  2000 error value =  0.4218545432998108
step =  4000 error value =  0.22953266761508415
step =  6000 error value =  0.15565873192166427
step =  8000 error value =  0.1171964920807123
step =  10000 error value =  0.09376374508993564
step =  12000 error value =  0.07804380381002565
step =  14000 error value =  0.06678840037612917
step =  16000 error value =  0.05834218496056356
step =  18000 error value =  0.05177534606163708
step =  20000 error value =  0.04652631276396696

Elapsed Time =>  0:00:09.138809


In [6]:
# OR Gate prediction
print(OR_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 1])

accuracy_ret = OR_obj.accuracy(test_xdata, test_tdata)

print("Accuracy => ", accuracy_ret)

OR_GATE 

Accuracy =>  1.0


In [7]:
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

NAND_obj = LogicGate("NAND_GATE", xdata, tdata, 1e-2, 20001)

NAND_obj.train()

Initial error value =  2.851777997845911
step =  0 error value =  2.842580925611612
step =  2000 error value =  0.6724796392287391
step =  4000 error value =  0.3946143311138737
step =  6000 error value =  0.27710237416641287
step =  8000 error value =  0.21256924207828853
step =  10000 error value =  0.17198813410411976
step =  12000 error value =  0.14420347943316972
step =  14000 error value =  0.124030007125247
step =  16000 error value =  0.10873890014876089
step =  18000 error value =  0.09676100302221496
step =  20000 error value =  0.08713167731615835

Elapsed Time =>  0:00:10.254263


In [9]:
# NAND Gate prediction
print(NAND_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 1, 1, 1, 0])

accuracy_ret = NAND_obj.accuracy(test_xdata, test_tdata)

print("Accuracy => ", accuracy_ret)

NAND_GATE 

Accuracy =>  1.0


In [11]:
# XOR Gate prediction
xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0])

XOR_obj = LogicGate("XOR_GATE", xdata, tdata, 1e-2, 20001)

XOR_obj.train()

Initial error value =  3.1594986259438187
step =  0 error value =  3.149966312857644
step =  2000 error value =  2.7726168604770605
step =  4000 error value =  2.772587972676928
step =  6000 error value =  2.77258792233199
step =  8000 error value =  2.77258792224003
step =  10000 error value =  2.772587922239862
step =  12000 error value =  2.7725879222398615
step =  14000 error value =  2.7725879222398615
step =  16000 error value =  2.7725879222398615
step =  18000 error value =  2.7725879222398615
step =  20000 error value =  2.7725879222398615

Elapsed Time =>  0:00:09.282586


In [12]:
# XOR Gate prediction
print(XOR_obj.name, "\n")

test_xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
test_tdata = np.array([ 0, 1, 1, 0])

accuracy_ret = XOR_obj.accuracy(test_xdata, test_tdata)

print("Accuracy => ", accuracy_ret)

XOR_GATE 

Accuracy =>  0.5
